In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from skimage import io, transform
from tqdm import tqdm
import cv2
import tensorflow as tf
from tensorflow.keras import Sequential,layers
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

C:\Users\adity\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
training_size=149
testing_size=50
training_file_name=np.array(['./training_data/images/ ('+str(i)+').png' for i in range(1,training_size+1)])
testing_file_name=np.array(['./testing_data/images/ ('+str(i)+').png' for i in range(1,testing_size+1)])

## Preprocessing

In [3]:
class metaDataLoader:
    def __init__(self):
        pass
    
    def blurFft(self,image):
        size=60
        (h, w) = image.shape
        (cX, cY) = (int(w / 2.0), int(h / 2.0))
        fft = np.fft.fft2(image)
        fftShift = np.fft.fftshift(fft)
        fftShift[cY - size:cY + size, cX - size:cX + size] = 0
        fftShift = np.fft.ifftshift(fftShift)
        recon = np.fft.ifft2(fftShift)
        magnitude = 20 * np.log(np.abs(recon))
        mean = np.mean(magnitude)
        return mean
    
    def blurLaplacian(self,image):
        return cv2.Laplacian(image, cv2.CV_64F).var()
    
    def morphologicalDilation(self,img):
        if(np.max(img)<=1):
            img=np.uint8(img*255)
        img[img<=205]=0
        img[img>205]=255
        th = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        black = np.zeros([img.shape[0] + 2, img.shape[1] + 2], np.uint8)
        mask = cv2.floodFill(th.copy(), black, (0,0), 0, 0, 0, flags=8)[1]
        mask = cv2.floodFill(mask.copy(), black, (img.shape[1]-1,img.shape[0]-1), 0, 0, 0, flags=8)[1]
        mask = cv2.floodFill(mask.copy(), black, (0,img.shape[0]-1), 0, 0, 0, flags=8)[1]
        mask = cv2.floodFill(mask.copy(), black, (img.shape[1]-1,0), 0, 0, 0, flags=8)[1]
        kernel_length = 30
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
        dilate = cv2.dilate(mask, horizontal_kernel, iterations=1)
        return dilate
    
    def blurMeta(self, image):
        return np.array([self.blurFft(image),self.blurLaplacian(image)])

    def rotationMeta(self,image):
        dilate=self.morphologicalDilation(image)
        tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)
        h, theta, d = transform.hough_line(dilate, theta=tested_angles)
        _,angles,_=transform.hough_line_peaks(h, theta, d)
        angles=[i*180/np.pi for i in angles]
        pred_angle=90-abs(np.median(angles))
        return np.array([pred_angle if np.median(angles)>0 else -1*pred_angle])
        
    def cropMeta(self,image,angle):
        image=transform.rotate(image,-1*angle[0])
        image=np.uint8(image*255)
        dilate=self.morphologicalDilation(image)
        contours=cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours=contours[0] if len(contours) == 2 else contours[1]
        bounding_rects=[cv2.boundingRect(i) for i in contours]
        if len(bounding_rects)!=0:
            areas=[i[2]*i[3] for i in bounding_rects]
            sorted_areas=sorted(areas)
            x=-1
            if sorted_areas[-1]==image.shape[0]*image.shape[1]:
                x=-2
            rect=bounding_rects[areas.index(sorted_areas[x])]
            meta_data=[rect[0],rect[1],image.shape[0]-rect[0]-rect[2],image.shape[1]-rect[1]-rect[3]]
        else:
            meta_data=[-1,-1,-1,-1]
        return np.array(meta_data)
    
    def getMetaData(self,image):
        if(len(image.shape)>2):
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurMetaData=self.blurMeta(image)
        rotMetaData=self.rotationMeta(image)
        cropMetaData=self.cropMeta(image,rotMetaData)
        return {
            'Blur':blurMetaData,
            'Rotation':rotMetaData,
            'Crop':cropMetaData
        }

In [4]:
class dataLoader(metaDataLoader):
    def __init__(self):
        super().__init__()
        pass
    
    def imageAugmentation(self,img):
        shape=img.shape
        blur_filter_size=np.random.randint(10,50,2)
        img=cv2.blur(img,blur_filter_size) 
        rotation_angle=(np.random.randint(0,1)-1)*(np.random.randint(15,90))
        img=transform.rotate(img,rotation_angle)
        size=[np.random.randint(shape[0]//2,shape[0]),np.random.randint(shape[1]//2,shape[1])]
        img=tf.image.random_crop(img, size).numpy()
        return img
    
    def loadImage(self,file_name,resize=None):
        image=io.imread(file_name)
        if resize:
            image=transform.resize(image,output_shape=resize)
        return image
    
    def generateDataset(self,file_names,resize=None):
        splited_dataset=train_test_split(file_names,test_size=.5)
        labels=np.concatenate([np.zeros(len(splited_dataset[0])),np.ones(len(splited_dataset[1]))])
        features=[]
        splited_dataset=np.concatenate(splited_dataset)
        for i in tqdm(range(len(splited_dataset))):
            image=self.loadImage(splited_dataset[i],resize)
            if labels[i]==1:
                image=self.imageAugmentation(image)
            features.append(np.concatenate(list(self.getMetaData(image).values())))
        features,labels=shuffle(features,labels)
        features=np.array(features)                         
        labels=np.array(labels)
        return features,labels

## Loading training and testing data

In [5]:
loader=dataLoader()

In [6]:
train_features,train_labels=loader.generateDataset(training_file_name)

 86%|████████████████████████████████████████████████████████████████████▋           | 128/149 [03:47<00:30,  1.47s/it]C:\Users\adity\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\adity\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\adity\AppData\Local\Temp\ipykernel_2988\1774208387.py:51: RuntimeWarning: invalid value encountered in cast
  image=np.uint8(image*255)
100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [04:06<00:00,  1.65s/it]


In [7]:
test_features,test_labels=loader.generateDataset(testing_file_name)

 86%|██████████████████████████████████████████████████████████████████████▌           | 43/50 [01:20<00:08,  1.20s/it]C:\Users\adity\AppData\Local\Temp\ipykernel_2988\1774208387.py:51: RuntimeWarning: invalid value encountered in cast
  image=np.uint8(image*255)
100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:27<00:00,  1.74s/it]


In [8]:
train_features=np.nan_to_num(train_features)
test_features=np.nan_to_num(test_features)

## Simple Neural Network

In [9]:
model=Sequential([
    layers.Dense(7,input_shape=(7,),activation='linear'),
    layers.Dropout(0.5),
    layers.Dense(3,activation='linear'),
    layers.Dropout(0.5),
    layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) 

In [10]:
model.fit(train_features,train_labels,epochs=100)

Epoch 1/100
5/5 [==============================] - 3s 10ms/step - loss: 1056.0287 - accuracy: 0.3893
Epoch 2/100
5/5 [==============================] - 0s 5ms/step - loss: 978.7627 - accuracy: 0.3826
Epoch 3/100
5/5 [==============================] - 0s 10ms/step - loss: 1011.9347 - accuracy: 0.4228
Epoch 4/100
5/5 [==============================] - 0s 6ms/step - loss: 789.2164 - accuracy: 0.4027
Epoch 5/100
5/5 [==============================] - 0s 8ms/step - loss: 753.0505 - accuracy: 0.3691
Epoch 6/100
5/5 [==============================] - 0s 6ms/step - loss: 759.4157 - accuracy: 0.4295
Epoch 7/100
5/5 [==============================] - 0s 6ms/step - loss: 710.9442 - accuracy: 0.4899
Epoch 8/100
5/5 [==============================] - 0s 6ms/step - loss: 673.0383 - accuracy: 0.4295
Epoch 9/100
5/5 [==============================] - 0s 6ms/step - loss: 757.0835 - accuracy: 0.4899
Epoch 10/100
5/5 [==============================] - 0s 7ms/step - loss: 658.2606 - accuracy: 0.4698
Epoch

5/5 [==============================] - 0s 5ms/step - loss: 89.5782 - accuracy: 0.7517
Epoch 84/100
5/5 [==============================] - 0s 5ms/step - loss: 76.8736 - accuracy: 0.7047
Epoch 85/100
5/5 [==============================] - 0s 5ms/step - loss: 14.0673 - accuracy: 0.8188
Epoch 86/100
5/5 [==============================] - 0s 5ms/step - loss: 79.8832 - accuracy: 0.7785
Epoch 87/100
5/5 [==============================] - 0s 5ms/step - loss: 57.0380 - accuracy: 0.7718
Epoch 88/100
5/5 [==============================] - 0s 6ms/step - loss: 45.1502 - accuracy: 0.7852
Epoch 89/100
5/5 [==============================] - 0s 6ms/step - loss: 32.1468 - accuracy: 0.7651
Epoch 90/100
5/5 [==============================] - 0s 6ms/step - loss: 77.0012 - accuracy: 0.7517
Epoch 91/100
5/5 [==============================] - 0s 6ms/step - loss: 86.6491 - accuracy: 0.7315
Epoch 92/100
5/5 [==============================] - 0s 5ms/step - loss: 51.1149 - accuracy: 0.8255
Epoch 93/100
5/5 [=====

In [11]:
model.evaluate(test_features,test_labels)

2/2 [==============================] - 1s 18ms/step - loss: 6.9607e-13 - accuracy: 1.0000


[6.960704256910033e-13, 1.0]

In [12]:
model_pred=model.predict(test_features)

2/2 [==============================] - 0s 5ms/step


In [13]:
model_pred=[int(i[0]) for i in model_pred]

In [14]:
print(classification_report(test_labels,model_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        25
         1.0       1.00      1.00      1.00        25

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50



## SVM Classifier

In [15]:
sv=SVC()
sv.fit(train_features,train_labels)

SVC()

In [16]:
pred=sv.predict(test_features)

In [17]:
print(classification_report(test_labels,pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        25
         1.0       1.00      1.00      1.00        25

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50



## Decision Tree Classifier

In [18]:
tree=DecisionTreeClassifier()

In [19]:
tree.fit(train_features,train_labels)

DecisionTreeClassifier()

In [20]:
pred2=tree.predict(test_features)

In [21]:
print(classification_report(test_labels,pred2))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        25
         1.0       1.00      1.00      1.00        25

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50



## Saving models

In [22]:
model.save("ANN.keras")

In [23]:
import joblib

In [24]:
joblib.dump(sv,"SVC.pkl")

['SVC.pkl']

In [25]:
joblib.dump(tree,"DTC.pkl")

['DTC.pkl']